In [1]:
from robustbench.data import load_cifar10
from robustbench.data import load_cifar10c
from robustbench.utils import clean_accuracy
from robustbench.utils import load_model
import foolbox as fb
from autoattack import AutoAttack
import torch

In [2]:
def pgd_attack(model_name:str, dataset:str, threat_model:str):
    x_test, y_test = load_cifar10(n_examples=50)
    model = load_model(model_name=model_name, dataset=dataset, threat_model=threat_model)
    fmodel = fb.PyTorchModel(model, bounds=(0, 1))
    _, advs, success = fb.attacks.LinfPGD()(fmodel, x_test.to('cuda:0'), y_test.to('cuda:0'), epsilons=[8/255])
    print('Robust accuracy: {:.1%}'.format(1 - success.float().mean()))

In [3]:
def auto_attack(model_name:str, dataset:str, threat_model:str, attacks:list): 
    x_test, y_test = load_cifar10(n_examples=50)
    model = load_model(model_name=model_name, dataset=dataset, threat_model=threat_model)
    if torch.cuda.is_available():
        model.cuda()
    adversary = AutoAttack(model, norm='Linf', eps=8/255, version='custom', attacks_to_run=attacks)
    adversary.apgd.n_restarts = 1
    x_adv = adversary.run_standard_evaluation(x_test, y_test)

In [4]:

def corruption_attack(corruption_type:str, model_names:list, dataset:str, threat_model:str):
    corruptions = [corruption_type]
    x_test, y_test = load_cifar10c(n_examples=1000, corruptions=corruptions, severity=5)
    for model_name in model_names:
        model = load_model(model_name, dataset='cifar10', threat_model=threat_model)
        acc = clean_accuracy(model, x_test, y_test)
        print(f'Model: {model_name}, CIFAR-10-C accuracy: {acc:.1%}')

In [5]:
pgd_attack(model_name='Carmon2019Unlabeled',dataset='cifar10',threat_model='Linf')

Files already downloaded and verified
Robust accuracy: 62.0%


In [6]:
list_of_model_names = ['Standard', 'Engstrom2019Robustness'] #, 'Rice2020Overfitting','Carmon2019Unlabeled']
corruption_attack(corruption_type='fog',model_names=list_of_model_names,dataset='cifar10',threat_model='Linf')

Model: Standard, CIFAR-10-C accuracy: 74.4%
Model: Engstrom2019Robustness, CIFAR-10-C accuracy: 38.8%


In [7]:
list_of_attacks = ['apgd-ce', 'apgd-dlr']
auto_attack(model_name='Carmon2019Unlabeled',dataset='cifar10',threat_model='Linf',attacks=list_of_attacks)

Files already downloaded and verified
using custom version including apgd-ce, apgd-dlr
initial accuracy: 92.00%
apgd-ce - 1/1 - 19 out of 46 successfully perturbed
robust accuracy after APGD-CE: 54.00% (total time 38.8 s)
apgd-dlr - 1/1 - 1 out of 27 successfully perturbed
robust accuracy after APGD-DLR: 52.00% (total time 62.8 s)
max Linf perturbation: 0.03137, nan in tensor: 0, max: 1.00000, min: 0.00000
robust accuracy: 52.00%


In [8]:
torch.cuda.empty_cache()